HW4 - Fake News Detection
本次作業將針對假新聞作分析，預測一則新聞是否reliable (繳交期限：12/10 23:55)
1: fake
0: true
請分別利用GBDT、LightGBM、xgboost對"train.csv"的資料建模，並用"test.csv"進行測試
註："test.csv"的label在"sample_submission.csv"裡面
(資料來源：https://www.kaggle.com/c/fakenewskdd2020/data)

作業流程：

1. 資料前處理
 a. 讀取"train.csv"與"test.csv"並利用分割符號切割、建立train&test之DataFrame
註：分割符號為tab(\t)

 b. 去除停頓詞stop words 
可參考：
sklearn.feature_extraction.text.CountVectorizer
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

自訂stop words
https://stackoverflow.com/questions/52712254/how-to-eliminate-stop-words-only-using-scikit-learn

 c. 文字探勘前處理，將文字轉換成向量，像是常見的方法 tf-idf、word2vec...等

可參考：
sklearn.feature_extraction.text.TfidfVectorizer
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text

Word2vec
https://radimrehurek.com/gensim/models/word2vec.html

2. 建模：分別使用以下三種模型
xgboost
https://xgboost.readthedocs.io/en/latest/python/python_intro.html#install-xgboost

GBDT

LightGBM
https://machinelearningmastery.com/gradient-boosting-with-scikit-learn-xgboost-lightgbm-and-catboost/

3. 評估模型
利用"test.csv"的資料對2.所建立的模型進行測試，並計算Accuracy、Precision、Recall、F-measure

請將以下檔案壓縮成 HW4_學號_姓名.zip：

1. 討論報告PDF (含截圖跟簡短說明、討論等等)，需提及：
文字探勘前處理的方法
GBDT、LightGBM、xgboost 模型之結果比較

2. Python程式碼

In [10]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.csv", sep='\t', encoding='utf-8')
test = pd.read_csv("test.csv", sep='\t', encoding='utf-8')
test_label = pd.read_csv("sample_submission.csv", sep='\t', encoding='utf-8')
train.head()

# type(train) : pandas.core.frame.DataFrame
# train.columns : Index(['text', 'label'], dtype='object')
# print( len(train['text'])): 4987
# print(len(test['text'])): 1247

,text,label
0,Get the latest from TODAY Sign up for our news...,1
1,2d Conan On The Funeral Trump Will Be Invited...,1
2,It’s safe to say that Instagram Stories has fa...,0
3,Much like a certain Amazon goddess with a lass...,0
4,At a time when the perfect outfit is just one ...,0


In [5]:
test_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id,label  1247 non-null   object
dtypes: object(1)
memory usage: 9.9+ KB


In [11]:
# 把 sample_submission.csv 中的 id 與 label切開，再塞入 test

label=[]
for i in range(len(test_label["id,label"])):
    temp_label = test_label["id,label"][i][-1]
    label.append(int(temp_label))
    
test_label["label"]=label

test["label"]= test_label["label"]
test

,id,text,label
0,2,The 2017 Teen Choice Awards ceremony was held ...,1
1,3,"The concert, part of “The Joshua Tree Tour,” w...",1
2,4,Selena Gomez refuses to talk to her mother abo...,0
3,5,This is worse than a lump of coal in your stoc...,0
4,6,Luann De Lesseps is going to rehab after her a...,0
...,...,...,...
1242,1244,Get the latest from TODAY Sign up for our news...,0
1243,1245,Jaden Smith claims that the Four Seasons Hotel...,0
1244,1246,Overview (3) Mini Bio (1) Faith Hill was bor...,1
1245,1247,CLOSE Aaron Paul dishes on 'The Path' Aaron P...,1


In [10]:
train["text"][0]

'Get the latest from TODAY Sign up for our newsletter  No one ever truly gets over losing a loved one, and Blake Shelton is no exception. He was just 14 when his older brother Richie died on Nov. 13, 1990. And, as Shelton noted in a tweet Monday, "It changed my life forever."  Richie was 24 when he died in a car accident in the Sheltons\' home state of Oklahoma. Two years ago, Shelton sent out a message for the 25th anniversary of his loss:  Richie, who was Blake\'s half-brother (they shared a mother), was a passenger in a car that collided with a school bus in Ada, south of Oklahoma City.  Richie, driver Redena McManus and a 3-year-old boy, Christopher McManus, all died during or shortly after the collision, while the bus driver and passengers were uninjured, according to police reports.  The accident has clearly remained with Blake, who told 60 Minutes in 2014, "I remember picking up the phone to call him a week after he was dead, to tell him something. I was picking up the phone to 

In [209]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4987 entries, 0 to 4986
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4987 non-null   object
 1   label   4987 non-null   object
dtypes: object(2)
memory usage: 78.0+ KB


In [210]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1247 entries, 0 to 1246
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1247 non-null   int64 
 1   text    1247 non-null   object
 2   label   1247 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 29.4+ KB


In [12]:
# 有個 label夾在0.1裡面，抓出來刪掉row1615
# train["label"].value_counts()
# 0        2972
# 1        2014
# label       1
# Name: label, dtype: int64


# a=0
# for i in train["label"]:
#     if i!="label":
#         a+=1
#     if i=="label":
#         break
# a=1615
train = train.drop([1615])

In [4]:
train["label"].value_counts()

0    2972
1    2014
Name: label, dtype: int64

In [13]:

# scikit learn的vecrizor+tfidftransformer 的方法做文字-數字化-向量化的結果放到模型會出現Error，但是用林大貴書內的方法擇可以順利套用模型，因此使用：
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

token = Tokenizer(num_words=3800)
token.fit_on_texts(train["text"])

In [14]:
x_train_seq = token.texts_to_sequences(train["text"])
x_test_seq  = token.texts_to_sequences(test["text"])

In [15]:
x_train = sequence.pad_sequences(x_train_seq, maxlen=2500)
x_test  = sequence.pad_sequences(x_test_seq,  maxlen=2500)

In [16]:
# prepare train & test data for xgboost
train_X = x_train
train_y = train["label"].values
test_X = x_test
test_y = test["label"].values

In [17]:
"""
xgboost
"""
import xgboost as xgb

# read in data
dtrain = xgb.DMatrix(train_X , label=train_y)
dtest = xgb.DMatrix(test_X , label=test_y)
# specify parameters via map
param = {'max_depth': 100, 'eta': 1, 'objective': 'reg:logistic'}
param['nthread'] = 4
#param['eval_metric'] = 'auc'

num_round = 4 # 越靠近3或4，模型評分越好。
bst = xgb.train(param, dtrain, num_round)

#After training, the model can be saved.
bst.save_model('0001.model')

#A saved model can be loaded as follows:
bst = xgb.Booster({'nthread': 4})  # init model
bst.load_model('0001.model')  # load data

# make prediction
y_pred = bst.predict(dtest)


# 計算Precision, Recall, Accuracy
# II. 函式計算
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
precision = precision_score(test_y,y_pred.round())
recall = recall_score(test_y,y_pred.round())
accuracy = accuracy_score(test_y,y_pred.round())
f1_measure = f1_score(test_y,y_pred.round())
print("XGboost precision=",precision)
print("XGboost recall=",recall)
print("XGboost accuracy=",accuracy)
print("XGboost f1_score=",f1_measure)


XGboost precision= 0.4965986394557823
XGboost recall= 0.3549432739059968
XGboost accuracy= 0.5028067361668003
XGboost f1_score= 0.41398865784499056


In [ ]:
import matplotlib
#畫出 XGB的樹
xgb.to_graphviz(bst)

In [ ]:
# 用countvectorize + Tfidftransformer的最佳結果:
# XGboost precision= 0.5138504155124654
# XGboost recall= 0.6012965964343598
# XGboost accuracy= 0.5212510024057738
# XGboost f1_score= 0.5541448842419716

# using only CountVectorizer, not using TfidfTransformer:
# XGboost precision= 0.47985347985347987
# XGboost recall= 0.21231766612641814
# XGboost accuracy= 0.4963913392141139
# XGboost f1_score= 0.2943820224719101

# param = {'max_depth': 25, 'eta': 1, 'objective': 'reg:logistic'}, param['nthread'] = 4, num_round = 2
# XGboost precision= 0.5114942528735632
# XGboost recall= 0.5769854132901134
# XGboost accuracy= 0.5180433039294307
# XGboost f1_score= 0.5422696115765423

# 'max_depth': 25→10
# XGboost precision= 0.5278969957081545
# XGboost recall= 0.3987034035656402
# XGboost accuracy= 0.5260625501202887
# XGboost f1_score= 0.45429362880886426

# 'max_depth': 25→100
# XGboost precision= 0.5080645161290323
# XGboost recall= 0.6126418152350082
# XGboost accuracy= 0.5148356054530874
# XGboost f1_score= 0.5554739162380602

# 'max_depth': 25→200→可見100已達飽和點
# XGboost precision= 0.5080645161290323
# XGboost recall= 0.6126418152350082
# XGboost accuracy= 0.5148356054530874
# XGboost f1_score= 0.5554739162380602

# 確定將'max_depth'設定為100之後：
# 'eta':1→3，評分全面下降，確定將 'eta'設為1
# XGboost precision= 0.4692005242463958
# XGboost recall= 0.580226904376013
# XGboost accuracy= 0.46752205292702487
# XGboost f1_score= 0.518840579710145

# param['nthread'] = 4→10：評分完全沒變，所以維持用4
# XGboost precision= 0.5080645161290323
# XGboost recall= 0.6126418152350082
# XGboost accuracy= 0.5148356054530874
# XGboost f1_score= 0.5554739162380602

# num_round = 2→6
# XGboost precision= 0.4978038067349927
# XGboost recall= 0.5510534846029174
# XGboost accuracy= 0.5028067361668003
# XGboost f1_score= 0.523076923076923

# num_round = 2→5
# XGboost precision= 0.502906976744186
# XGboost recall= 0.5607779578606159
# XGboost accuracy= 0.5084202085004009
# XGboost f1_score= 0.5302681992337166

# num_round = 2→4
# XGboost precision= 0.5138504155124654
# XGboost recall= 0.6012965964343598
# XGboost accuracy= 0.5212510024057738
# XGboost f1_score= 0.5541448842419716

# num_round = 2→3
# XGboost precision= 0.5026809651474531
# XGboost recall= 0.6077795786061588
# XGboost accuracy= 0.5084202085004009
# XGboost f1_score= 0.5502567865003669

In [115]:
# GBDT need y to be int.
train_y=[]
for i in train["label"]:
    a = int(i)
    train_y.append(a)

In [116]:
# prepare train & test data for GBDT
train_X = train_vector
train_y = train_y
test_X = test_vector
test_y = test["label"].values

In [117]:
"""
GBDT
"""
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(learning_rate=5.0, max_depth=5, random_state=43)
clf.fit(train_X, train_y)
y_pred_GBDT=clf.predict(test_X)

# 計算Precision, Recall, Accuracy
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

precision = precision_score(test_y,y_pred_GBDT.round())
recall = recall_score(test_y,y_pred_GBDT.round())
accuracy = accuracy_score(test_y,y_pred_GBDT.round())
f1_measure = f1_score(test_y,y_pred_GBDT.round())
print("GBDT precision=",precision)
print("GBDT recall=",recall)
print("GBDT accuracy=",accuracy)
print("GBDT f1_score=",f1_measure)

GBDT precision= 0.49617672047578587
GBDT recall= 0.946515397082658
GBDT accuracy= 0.49799518845228546
GBDT f1_score= 0.6510590858416945


In [ ]:
# learning_rate=1.0, max_depth=1, random_state=0, n_estimators=100, subsample=1.0
# GBDT precision= 0.5386416861826698
# GBDT recall= 0.3727714748784441
# GBDT accuracy= 0.5316760224538893
# GBDT f1_score= 0.4406130268199234

# subsample 1.0→0.5 ：沒有很大幫助
# GBDT precision= 0.49714285714285716
# GBDT recall= 0.42301458670988656
# GBDT accuracy= 0.5028067361668003
# GBDT f1_score= 0.45709281961471104

# learning_rate=5.0, max_depth=5, random_state=0, n_estimators=100
# GBDT precision= 0.4953350296861747
# GBDT recall= 0.946515397082658
# GBDT accuracy= 0.4963913392141139
# GBDT f1_score= 0.6503340757238306

# random_state=0→43，為什麼random_state=0 → 43； precision和accuracy會上升?
# GBDT precision= 0.49617672047578587
# GBDT recall= 0.946515397082658
# GBDT accuracy= 0.49799518845228546
# GBDT f1_score= 0.6510590858416945

# learning_rate=1.0, max_depth=5, random_state=43, n_estimators=500， n_estimators放大五倍，不如learning_rate放大五倍
# GBDT precision= 0.4888888888888889
# GBDT recall= 0.39222042139384117
# GBDT accuracy= 0.4963913392141139
# GBDT f1_score= 0.43525179856115104

# learning_rate=1.0, max_depth=10, random_state=0, n_estimators=100
# GBDT precision= 0.5284090909090909
# GBDT recall= 0.4521880064829822
# GBDT accuracy= 0.529270248596632
# GBDT f1_score= 0.4873362445414848

# learning_rate=5.0, max_depth=10, random_state=0, n_estimators=100
# GBDT precision= 0.4831981460023175
# GBDT recall= 0.6758508914100486
# GBDT accuracy= 0.48195669607056935
# GBDT f1_score= 0.5635135135135135

# learning_rate=5.0, max_depth=6, random_state=0, n_estimators=100
# GBDT precision= 0.4919695688926458
# GBDT recall= 0.9432739059967585
# GBDT accuracy= 0.48997594226142743
# GBDT f1_score= 0.6466666666666667

# learning_rate=3.0, max_depth=6, random_state=0, n_estimators=100
# GBDT precision= 0.5348189415041783
# GBDT recall= 0.31118314424635335
# GBDT accuracy= 0.5252606255012029
# GBDT f1_score= 0.39344262295081966

# learning_rate=5.0, max_depth=4, random_state=0, n_estimators=100
# GBDT precision= 0.4941275167785235
# GBDT recall= 0.9546191247974068
# GBDT accuracy= 0.49398556535685645
# GBDT f1_score= 0.6511885019347705

# learning_rate=5.0, max_depth=4, random_state=0, n_estimators=500：learning_rate=0 讓每顆小樹權重備縮小，所以500顆樹跟100顆樹表現一樣
# GBDT precision= 0.4941275167785235
# GBDT recall= 0.9546191247974068
# GBDT accuracy= 0.49398556535685645
# GBDT f1_score= 0.6511885019347705

# learning_rate=5→4.0
# GBDT precision= 0.4792099792099792
# GBDT recall= 0.747163695299838
# GBDT accuracy= 0.4731355252606255
# GBDT f1_score= 0.5839138695376821

# learning_rate=5→6
# GBDT precision= 0.4804772234273319
# GBDT recall= 0.7179902755267423
# GBDT accuracy= 0.47634322373696875
# GBDT f1_score= 0.575698505523067

In [118]:
# prepare train & test data for LightGBM
train_X = train_vector
train_y = train["label"].values
test_X = test_vector
test_y = test["label"].values

In [119]:
"""
Light GBM
"""

import lightgbm as lgb 

train_data = lgb.Dataset(train_X , label=train_y)

param = {'num_leaves': 50, 'objective': 'binary'}
param['metric'] = 'auc'

num_round = 100
bst = lgb.train(param, train_data, num_round)

# After training, the model can be saved:
bst.save_model('model.txt')

# A saved model can be loaded:
bst = lgb.Booster(model_file='model.txt')  # init model

ypred_LightGBM = bst.predict(test_X, predict_disable_shape_check="true")

[LightGBM] [Info] Number of positive: 2014, number of negative: 2972
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031050 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 209339
[LightGBM] [Info] Number of data points in the train set: 4986, number of used features: 2482
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.403931 -> initscore=-0.389112
[LightGBM] [Info] Start training from score -0.389112


In [120]:
# 計算Precision, Recall, Accuracy
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

precision_LightGBM = precision_score(test_y,ypred_LightGBM.round())
recall_LightGBM = recall_score(test_y,ypred_LightGBM.round())
accuracy_LightGBM = accuracy_score(test_y,ypred_LightGBM.round())
f1_measure_LightGBM = f1_score(test_y,ypred_LightGBM.round())
print("Light GBM precision=",precision_LightGBM)
print("Light GBM recall=",recall_LightGBM)
print("Light GBM accuracy=",accuracy_LightGBM)
print("Light GBM f1_score=",f1_measure_LightGBM)
# 加入Tfidftransform之前
# Light GBM precision= 0.49390243902439024
# Light GBM recall= 0.1312803889789303
# Light GBM accuracy= 0.5036086607858862
# Light GBM f1_score= 0.20742637644046094

# param = {'num_leaves': 31,
#num_round = 10
# Light GBM precision= 0.5384615384615384
# Light GBM recall= 0.23824959481361427
# Light GBM accuracy= 0.5220529270248596
# Light GBM f1_score= 0.33033707865168543

# param = {'num_leaves': 60,
#num_round = 10
# Light GBM precision= 0.5047619047619047
# Light GBM recall= 0.2576985413290113
# Light GBM accuracy= 0.5076182838813151
# Light GBM f1_score= 0.3412017167381974

# param = {'num_leaves': 50, 
# num_round = 10
# Light GBM precision= 0.5317220543806647
# Light GBM recall= 0.2852512155591572
# Light GBM accuracy= 0.5220529270248596
# Light GBM f1_score= 0.37130801687763715

# param = {'num_leaves': 50, 
# num_round = 50
# Light GBM precision= 0.5097276264591439
# Light GBM recall= 0.4246353322528363
# Light GBM accuracy= 0.5132317562149158
# Light GBM f1_score= 0.46330680813439434

# param = {'num_leaves': 50, 
# num_round = 100
# Light GBM precision= 0.5184466019417475
# Light GBM recall= 0.4327390599675851
# Light GBM accuracy= 0.520449077786688
# Light GBM f1_score= 0.4717314487632509


# Light GBM precision= 0.5184466019417475
# Light GBM recall= 0.4327390599675851
# Light GBM accuracy= 0.520449077786688
# Light GBM f1_score= 0.4717314487632509
# CounterVectorize有沒有加 tfidftransformer，模型精度是一樣的

Light GBM precision= 0.5184466019417475
Light GBM recall= 0.4327390599675851
Light GBM accuracy= 0.520449077786688
Light GBM f1_score= 0.4717314487632509
